In [ ]:
import pandas as pd
import nltk
import vaderSentiment
import re
import string

from collections import defaultdict
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
nltk.download('punkt')

Read Data from CSV File:

In [ ]:
path = r'C:\Users\JZ\Desktop\Data\JPMC_data_csv1.csv'

In [ ]:
def load():
    pd.set_option("display.max_colwidth", None) #-1
    return pd.read_csv(path)

In [ ]:
totalInfo = load().copy()

In [ ]:
totalInfo

In [ ]:
#fill any NaN/invalid values
totalInfo = totalInfo.fillna(0)

In [ ]:
#get rid of any noisy data (compound score is zero)
filter_noise = (totalInfo['compound score '] != '0')
totalInfo = totalInfo.loc[filter_noise]

In [ ]:
totalInfo

In [ ]:
# Filter special characters
def filter_special_chars(comment):
    new_sentence = '' #empty string
    new_comment = re.sub(r"https:\S+", "", comment) #remove url links from comment

    for i in range(0, len(new_comment)):
        if ord(new_comment[i]) >= 110000: #for characters like 𝐴𝑀𝐶 BB  𝑁𝑂𝐾 
             new_sentence += new_comment[i]
        if ord(new_comment[i]) <= 127: #keep all ascii characters
            new_sentence += new_comment[i]
                
    return new_sentence

In [ ]:
#IMPORTANT FINDINGS: chr(119886 - 119911) = 𝑎 - 𝑧 
#                    chr(119860 - 119885) = 𝐴 - 𝑍
#                    chr(119912 - 119937) = 𝑨 - 𝒁

#REGULAR chars       chr(97 - 122) = a - z
#                    chr(65 - 90)  = A- Z

In [ ]:
# chr_lower = {} #dictionary for lower characters
# reg_chr_upper = {} #dictionary for regular upper characters
# bold_chr_upper = {} #dictionary for bold upper characters

# lower = 119886
# cbold_upper = 119912
# c_upper = 119860

# reg_l = 97
# reg_u = 65

# for i in range(0, 26):
#     chr_lower[chr(lower)] = chr(reg_l)
#     bold_chr_upper[chr(cbold_upper)] = chr(reg_u)
#     reg_chr_upper[chr(c_upper)] = chr(reg_u)

#     lower += 1
#     cbold_upper += 1
#     c_upper += 1

#     reg_l += 1
#     reg_u += 1

In [ ]:
#print(chr_lower)

In [ ]:
#print(bold_chr_upper)

In [ ]:
#print(reg_chr_upper)

In [ ]:
# def convert_chr(comment): #Replace symbols with regular a-z or A-Z
#     new_comment = ''
#     match = False
#     for i in range(0, len(comment)):
#         match = False
#         if comment[i] in reg_chr_upper:
#             new_comment += reg_chr_upper.get(comment[i])
#             match = True
#         if comment[i] in bold_chr_upper:
#             new_comment += bold_chr_upper.get(comment[i])
#             match = True
#         if comment[i] in chr_lower:
#             new_comment += chr_lower.get(comment[i])
#             match = True
#         if match != True:
#             new_comment += comment[i]
#     return new_comment

In [ ]:
#convert_chr('Crush them! Good luck to  𝐴𝑀𝐶 GME we will soon unite $DWAC	')

Find if sentence has ... in it

In [ ]:
def remove_sentence(comment): #if text contains …, then it will be removed (… means truncated)
    obj = re.search('…', comment)
    if obj != None:
        return 1
    else:
        return -1
        

In [ ]:
totalInfo['remove'] = totalInfo['text'].map(lambda x: '1' if remove_sentence(x) == 1 else '-1')

Filter out sentences that are truncated:

In [ ]:
filter_truncated = totalInfo['remove'] == '-1'
full_comments_df = totalInfo.loc[filter_truncated]
full_comments_df

Filter out special characters for full comments:

In [ ]:
full_comments_df['new_text'] = full_comments_df['text'].map(lambda x: filter_special_chars(x)) # remove special char

In [ ]:
full_comments_df = full_comments_df[['Date','text','new_text','favorited', 'retweeted', 'lang', 'quoted_status', 'Stock_Name', 'sentiment', 'sentiment_class', 'compound score ', 'Stock Price', 'remove']]

In [ ]:
full_comments_df

In [ ]:
ord('𝑎')

In [ ]:
# def convert_chr(comment): #Replace symbols with regular a-z or A-Z
#     comment = str(comment)
#     chr_lower = {'𝑎': 'a', '𝑏': 'b', '𝑐': 'c', '𝑑': 'd', '𝑒': 'e', '𝑓': 'f', '𝑔': 'g', '\U0001d455': 'h', '𝑖': 'i', '𝑗': 'j', '𝑘': 'k', '𝑙': 'l', '𝑚': 'm', '𝑛': 'n', '𝑜': 'o', '𝑝': 'p', '𝑞': 'q', '𝑟': 'r', '𝑠': 's', '𝑡': 't', '𝑢': 'u', '𝑣': 'v', '𝑤': 'w', '𝑥': 'x', '𝑦': 'y', '𝑧': 'z'}
#     bold_chr_upper = {'𝑨': 'A', '𝑩': 'B', '𝑪': 'C', '𝑫': 'D', '𝑬': 'E', '𝑭': 'F', '𝑮': 'G', '𝑯': 'H', '𝑰': 'I', '𝑱': 'J', '𝑲': 'K', '𝑳': 'L', '𝑴': 'M', '𝑵': 'N', '𝑶': 'O', '𝑷': 'P', '𝑸': 'Q', '𝑹': 'R', '𝑺': 'S', '𝑻': 'T', '𝑼': 'U', '𝑽': 'V', '𝑾': 'W', '𝑿': 'X', '𝒀': 'Y', '𝒁':'Z'}
#     reg_chr_upper = {'𝐴': 'A', '𝐵': 'B', '𝐶': 'C', '𝐷': 'D', '𝐸': 'E', '𝐹': 'F', '𝐺': 'G', '𝐻': 'H', '𝐼': 'I', '𝐽': 'J', '𝐾': 'K', '𝐿': 'L', '𝑀': 'M', '𝑁': 'N', '𝑂': 'O', '𝑃': 'P', '𝑄': 'Q', '𝑅': 'R', '𝑆': 'S', '𝑇': 'T', '𝑈': 'U', '𝑉': 'V', '𝑊': 'W', '𝑋': 'X', '𝑌': 'Y', '𝑍': 'Z'}
#     new_comment = ''
#     for i in comment:
#         print(ord(i))
#         if ord(i) >= 119860 and ord(i) <= 119885:
#             new_comment += reg_chr_upper.get(i)
#             new_comment += 'adrian'
#         elif ord(i) >= 119912 and ord(i) <= 119937:
#             new_comment += bold_chr_upper.get(i)
#             new_comment += 'adrian'
#         elif ord(i) >= 119886 and ord(i) <= 119911:
#             new_comment += chr_lower.get(i)
#             new_comment += 'adrian'
#         else:
#             new_comment += i
#     return new_comment

In [ ]:
def convert_chr(comment): #Replace symbols with regular a-z or A-Z
    new_comment = ''
    for i in comment:
        num = ord(i)
        new_comment += chr(num)
    return new_comment

In [ ]:
full_comments_df['modified_text'] = full_comments_df['new_text'].apply(lambda x: convert_chr(x)) 

In [ ]:
full_comments_df[['text','new_text', 'modified_text']]

In [ ]:
full_comments_df['modified_final'] = full_comments_df['new_text'].apply(lambda x: convert_chr(x))
full_comments_df[['text','new_text','modified_final']]

In [ ]:
def add_adrian(comment):
    sentence = ' '
    for i in range(len(comment)):
        sentence += " " + str(ord(comment[i]))
        if ord(comment[i]) > 100000:
            return sentence
    return 0

In [ ]:
full_comments_df['modified_text'] = full_comments_df['new_text'].apply(lambda x: add_adrian(x))
full_comments_df

In [ ]:
full_comments_df

Modifed vaderSentiment Analyzer:

In [ ]:
stocks = totalInfo['Stock_Name'].to_numpy()
stocks

all_stocks = set(stocks)
all_stocks.remove('Stock_Name')

all_stocks

In [ ]:
def get_sentiment(comment):
    sentence_list = []
    stock_sentiment_dict = {}
    all_stocks_mentioned = defaultdict(list) # ex: GME:[1, 0.68]
                                             # 1 <-- number of times it appeared in comment
                                             # 0.68 <-- sum of compound sentimenent scores
                                             # Later find average of compound scores ex: 0.68/1 = 0.68 (average)
    avg_score = 0
    within_comment = nltk.sent_tokenize(comment) #makes a list of all sentences in comment
    
    for sentence in within_comment:
        stocks_seen = set() #within each sentence, keeps track of what stocks are seen
        analyzer = SentimentIntensityAnalyzer() #Create analyzer object
        vs = analyzer.polarity_scores(sentence) # sentiment of sentence
        words = nltk.word_tokenize(sentence) # makes a list of all the words within a sentence
        
        for word in words: #iterates through each word in sentence
            if word in all_stocks: #checks if stock name is a word
                if word not in all_stocks_mentioned: #adds word to dictionary if never seen before
                    if word not in stocks_seen:
                        stock_sentiment_dict[word] = 0
                        
                        stocks_seen.add(word)
                        all_stocks_mentioned[word].append(1)
                        all_stocks_mentioned[word].append(vs['compound'])
                
                else: #if word is already in dictionary (seen before)
                    if word not in stocks_seen:
                        all_stocks_mentioned[word][0] += 1
                        all_stocks_mentioned[word][1] += vs['compound']
            
    #calculate average compound score for each stock
    for name in all_stocks_mentioned.keys():
        stock_sentiment_dict[name] = all_stocks_mentioned[name][1] / all_stocks_mentioned[name][0]
    
    #put average stock sentiment in dictionary
    #stock_sentiment_dict[word] = avg_score
    print(all_stocks_mentioned)
    return stock_sentiment_dict

In [ ]:
a = "Crush them! Good luck to  𝐴𝑀𝐶 GME we will soon unite $DWAC"

In [ ]:
get_sentiment(a)

In [ ]:
"""### Sentiment Analyzer"""

def company_scores(sentence): #returns a dictionary with company names in interest with compound score 
    sentiment_dict = get_sentiment(sentence)
    return sentiment_dict


def apply_vadersentiment(df):
    df['modified']=df['new_text'].apply(lambda x: company_scores(x))
    #df['sentiment_class']=df['sentiment'].apply(lambda x: 'positive' if x['compound']>0.5 else ('negative' if x['compound']<-0.5 else 'neutral'))
    #df['compound score']=df['sentiment'].apply(lambda x: x['compound'])
    return df

Obtain all stocks that we are interested in:

In [ ]:
apply_vadersentiment(full_comments_df)